# Create the DataFrame from 1 json file.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json
data = json.load(open('/home/student/ttngo/SpotifyDataProject/data/mpd.slice.0-999.json'))
dfp = pd.DataFrame(data['playlists']) #get the dataframe from key of dictionary json

In [3]:
dfp

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37,NaN
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21,NaN
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31,NaN
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86,NaN
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,old,false,995,1507852800,41,40,1,"[{'pos': 0, 'artist_name': 'Katrina', 'track_u...",8,9917901,36,NaN
996,Daze,false,996,1479254400,17,17,1,"[{'pos': 0, 'artist_name': 'PARTYNEXTDOOR', 't...",13,3699248,15,NaN
997,rap,false,997,1410307200,119,98,1,"[{'pos': 0, 'artist_name': 'LoveRance', 'track...",63,27538723,82,NaN
998,Country,false,998,1507939200,108,75,1,"[{'pos': 0, 'artist_name': 'Hunter Hayes', 'tr...",37,24950143,40,NaN


## Create SQL table name playlist_name which contains only the column of 'name' in playlist dataframe.

In [16]:
import sqlite3
# Establish a connection to a database (it will create the database file if it doesn't exist)
conn = sqlite3.connect('playlists.db')

# Create a cursor object using the connection
cur = conn.cursor()

# From now, we can use the cursor to create table, insert value, retreive value from that
cur.execute('CREATE TABLE IF NOT EXISTS playlist_names (name TEXT)')

# Insert the names from the dataframe into the SQL table
dfp['name'].apply(lambda x: cur.execute('INSERT INTO playlist_names (name) VALUES (?)', (x,)))

# Commit the changes and close the connection
conn.commit()
conn.close()

In [17]:
# Re-establish the connection to the database
conn = sqlite3.connect('playlists.db')

# Create a cursor object using the connection
cur = conn.cursor()

# Execute a query to retrieve all rows from the playlist_names table
cur.execute('SELECT * FROM playlist_names')

# Fetch all results of the query
rows = cur.fetchall()

# Print the rows
for row in rows:
    print(row)

# Close the connection
conn.close()


('Throwbacks',)
('Awesome Playlist',)
('korean ',)
('mat',)
('90s',)
('Wedding',)
('I Put A Spell On You',)
('2017',)
('BOP',)
('old country ',)
('abby ',)
('VIBE',)
('relax',)
('sleep',)
("90's ",)
('New Songs',)
('slow hands',)
("Mom's playlist",)
('SARAH',)
('melancholy',)
('mixtape',)
('Sad Songs',)
("fall '17",)
('✔️',)
('Twenty one pilots',)
('run it',)
('Winter 2014',)
('smooth ',)
('Yeet',)
('groovy',)
('Garage Rock',)
('Running 2.0',)
('acoustic',)
('morning',)
('good good',)
('Good Vibes',)
(' indie rock',)
('Hits',)
('tb',)
('Country',)
('2020',)
('Belters',)
("Road Trippin'",)
('Country',)
('taylor swift',)
('angst',)
('Running 2',)
('Spring 2016',)
('Country',)
('Gospel',)
('SB',)
('Kevin',)
('Christmas',)
('NB',)
('Smooth Jazz',)
('Tennessee ',)
('funky',)
('spring 16',)
('Gates',)
('Favorite Songs',)
('Ski',)
('Country',)
('hype',)
('Awesome Mix',)
('Elizabeth',)
('PlayStation',)
('Ashley',)
('chill',)
('xx',)
('art class',)
('Random Tunes',)
('boat',)
('spanish jams',)


### Now , lets create playlists table(pid, name, ....), tracks table( posiition, artist_name, track_uri,....), track_playlist  table(playlist_id, track_id,pos), artists table(artist_uri, artst_name), albums table(album_id, album_name)

In [19]:
conn1 = sqlite3.connect('spotify.db')
cur1 = conn1.cursor()

### CREATE AND INSERT VALUE INTO "PLAYLIST" TABLE

**Note:** "create table if not exists" is the SQL command to create a new table onlyif it doesn't exist. (same name and same schemas)

In [20]:
#NOTE: This table is excluded the tracks columns.
create_playlist_table = '''
CREATE TABLE IF NOT EXISTS Playlists (
    pid INT PRIMARY KEY,
    name TEXT,
    collaborative TEXT,
    modified_at INT,
    num_tracks INT,
    num_albums INT,
    num_followers INT
    )
    '''
cur1.execute(create_playlist_table)


In [22]:
conn1.commit() #commit the change everytime we create, insert, update, delete,....

### INSERT VALUE OF INTO PLAYLIST FROM JSON FILE. 

In [24]:
# Prepare data for tracks insertion
playlists_data=[]
for playlist in data['playlists']:
    playlists_data.append((
        playlist['name'],
        playlist['collaborative'],
        playlist['pid'],
        playlist['modified_at'],
        playlist['num_tracks'],
        playlist['num_albums'],
        playlist['num_followers']
    ))

#Inserting each keys in the tracks_data into the table
insert_query = '''
INSERT OR IGNORE INTO Playlists (name, collaborative, pid, modified_at, num_tracks, num_albums, num_followers)
VALUES (?, ?, ?, ?, ?, ?, ?)
'''

cur1.executemany(insert_query, playlists_data)
conn1.commit()

### RETRIEVE THE DATABASE

**NOTE:** Everytime we retreive or modify the database. we have to reconnect to the db again. Retreiving db doesn't need to commit() but modifying the db need to commit() and closed the commit after.

In [26]:
# Re-establish the connection to the database
conn1 = sqlite3.connect('spotify.db')

# Create a cursor object using the connection
cur1 = conn1.cursor()

# Execute a query to retrieve all rows from the playlist_names table
cur1.execute('SELECT * FROM Playlists')

# Fetch all results of the query
rows = cur1.fetchall()

# Print the rows
for row in rows:
    print(row)

# Close the connection
cur1.close()
conn1.close()

('Throwbacks', 'false', 0, 1493424000, 52, 47, 1)
('Awesome Playlist', 'false', 1, 1506556800, 39, 23, 1)
('korean ', 'false', 2, 1505692800, 64, 51, 1)
('mat', 'false', 3, 1501027200, 126, 107, 1)
('90s', 'false', 4, 1401667200, 17, 16, 2)
('Wedding', 'false', 5, 1430956800, 80, 71, 1)
('I Put A Spell On You', 'false', 6, 1477094400, 16, 15, 1)
('2017', 'false', 7, 1509321600, 53, 52, 1)
('BOP', 'false', 8, 1508976000, 46, 37, 2)
('old country ', 'false', 9, 1501804800, 21, 20, 1)
('abby ', 'false', 10, 1509321600, 72, 60, 2)
('VIBE', 'false', 11, 1365379200, 134, 117, 1)
('relax', 'false', 12, 1481587200, 9, 9, 1)
('sleep', 'false', 13, 1509062400, 13, 13, 1)
("90's ", 'false', 14, 1382572800, 103, 79, 1)
('New Songs', 'false', 15, 1432684800, 7, 5, 1)
('slow hands', 'false', 16, 1491696000, 105, 94, 1)
("Mom's playlist", 'false', 17, 1465430400, 79, 69, 1)
('SARAH', 'false', 18, 1507334400, 68, 66, 1)
('melancholy', 'false', 19, 1430956800, 85, 82, 1)
('mixtape', 'false', 20, 150949

In [33]:
import sqlite3

# Establish the connection to the database
conn1 = sqlite3.connect('spotify.db')

# Create a cursor object using the connection
cur1 = conn1.cursor()

# Execute a query to count the number of rows in the Playlists table
cur1.execute('SELECT COUNT(*) FROM Playlists')

# Fetch the result of the query (it will be a single row with a single value)
row_count = cur1.fetchone()[0]

# Print the number of rows
print(f'Number of rows in the Playlists table: {row_count}')

# Close the cursor and connection
cur1.close()
conn1.close()


Number of rows in the Playlists table: 2000
